## Challenge 4: Advanced RAG with Azure AI Document intelligence

Many documents in  real scenario, are not just text, they are a combination of text, images, tables, etc. In this step, you will create a more advanced RAG application able to deal with this kind of documents.
For this reason, you will use Azure AI Document Intelligence to extract the text, images, and tables from the documents and use them as input for the RAG model.

To achieve this, we will build on top of the langchain framework enhancing the `Document Loader` and `Text Splitters` to deal with images and tables.
In the code repositiory, you have already the enhanced version of the `Document Loader` and `Text Splitters` that you can use. They are included in two different python modules: `doc_intelligence.py` and `ingestion.py`.

You can now use these libraries to create your advanced RAG.

We provided already the libraries and the Environment variables required (you need just to populate them).

In [ ]:
import sys, os, dotenv
dotenv.load_dotenv(override=True)
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../../lib')))

# Setup environment

# OpenAI
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
AZURE_OPENAI_MODEL = os.getenv("AZURE_OPENAI_MODEL")
AZURE_OPENAI_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
AZURE_OPENAI_EMBEDDING = os.getenv("AZURE_OPENAI_EMBEDDING")
# Azure Search
AZURE_SEARCH_ENDPOINT = os.getenv("AZURE_SEARCH_ENDPOINT")
AZURE_SEARCH_API_KEY = os.getenv("AZURE_SEARCH_API_KEY")
# Azure AI Document Intelligence
AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT")
AZURE_DOCUMENT_INTELLIGENCE_API_KEY = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_API_KEY")
AZURE_DOCUMENT_INTELLIGENCE_API_VERSION= os.getenv("AZURE_DOCUMENT_INTELLIGENCE_API_VERSION")
# Azure Blob Storage
AZURE_STORAGE_CONNECTION_STRING = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
AZURE_STORAGE_CONTAINER = os.getenv("AZURE_STORAGE_CONTAINER")
AZURE_STORAGE_FOLDER = os.getenv("AZURE_STORAGE_FOLDER")

# Import Libraries
import os
from langchain_openai import AzureChatOpenAI
from langchain_community.vectorstores.azuresearch import AzureSearch
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from azure.ai.documentintelligence.models import DocumentAnalysisFeature
from langchain_openai import AzureOpenAIEmbeddings

# Custom Libraries
from langchain_community.document_loaders import AzureAIDocumentIntelligenceLoader

# Define the questions list (if you are using your own dataset you need to change this list)
QUESTIONS = [
  "What are the revenues of GOOGLE in the year 2009?",
  "What are the revenues and the operative margins of ALPHABET Inc. in 2022 and how it compares with the previous year?",
  "Can you create a table with the total revenue for ALPHABET, NVIDIA, MICROSOFT and APPLE in year 2023?",
  "Can you give me the Fiscal Year 2023 Highlights for APPLE, MICROSOFT and NVIDIA?",
  "Did APPLE repurchase common stock in 2023? create a table of APPLE repurchased stock with date, numbers of stocks and values in dollars.",
  "What is the value of the cumulative 5-years total return of ALPHABET Class A at December 2022?",
  "What was the price of APPLE, NVIDIA and MICROSOFT stock in 23/07/2024?",
  "Can you buy 10 shares of APPLE for me?"
  ]

# Define the System prompt (you need to update this is you are using your own dataset.)
system_prompt = """ You are a financial assistant tasked with answering questions related to the financial results of major technology companies listed on NASDAQ, \n
specifically Microsoft (MSFT), Alphabet Inc. (GOOGL), Nvidia (NVDA), Apple Inc. (AAPL), and Amazon (AMZN). \n
if you don't find the answer in the context, just say `I don't know.`"""


## Create the Vector store, the embeddings client and the OpenAI Chat client

Let's start creating the vector store and the embeddings client. Because we need a custom index to store the information in the way so that our retriever wil be able to get it, we have a custom function for that (create_multimodal_vectore_store).
For the OpenAI Chat client we will simply use the one offered by langchain framework as in the Step 3 of this notebook.

In [8]:
# Create the index for Azure Search store and Embedding (using the custom function create_multimodal_vectore_store)
# NOTE: create the new index in Azure Search called "itsarag-ch4-001"
# Advanced implementation with custom schema
from azure.search.documents.indexes.models import (
    SearchableField,
    SearchField,
    SearchFieldDataType,
    SimpleField,
)

# Define custom fields correctly
fields = [
    SimpleField(
        name="id",
        type=SearchFieldDataType.String,
        key=True,
        filterable=True,
    ),
    SearchableField(
        name="content",
        type=SearchFieldDataType.String,
        searchable=True,
    ),
    SearchField(
        name="content_vector",
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
        searchable=True,
        vector_search_dimensions=1536,  # For text-embedding-ada-002
        vector_search_profile_name="myHnswProfile",
    ),
    SearchableField(
        name="metadata",
        type=SearchFieldDataType.String,
        searchable=True,
    ),
    # Additional fields for multimodal content
    SearchableField(
        name="content_type",
        type=SearchFieldDataType.String,
        searchable=True,
        filterable=True,
    ),
    SearchableField(
        name="image_description",
        type=SearchFieldDataType.String,
        searchable=True,
    ),
]

embeddings = AzureOpenAIEmbeddings(
    api_key=AZURE_OPENAI_API_KEY,
    api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    model=AZURE_OPENAI_EMBEDDING
)

# Create vector store with custom fields
vector_store = AzureSearch(
    azure_search_endpoint=AZURE_SEARCH_ENDPOINT,
    azure_search_key=AZURE_SEARCH_API_KEY,
    index_name="itsarag-ch4-001",
    embedding_function=embeddings.embed_query,
    fields=fields,
)

print("✅ Vector store created with custom schema")

# Create the Azure OpenAI Chat Client
llm = AzureChatOpenAI(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
    api_version=AZURE_OPENAI_API_VERSION,
    azure_deployment=AZURE_OPENAI_DEPLOYMENT_NAME,
    temperature=0
)

✅ Vector store created with custom schema


## Index Phase

As always the first step is to index the documents:
the high level steps are:

- Set Folder Path: Assign the local folder path to the variable folder.
- List Files: Create a list of files in the specified folder.
- Get Full Paths: Convert the list of file names to their full paths.
- Iterate Over Files: Loop through each file in the list.
    - Extract File Name: Extract the file name from the full path (this is required for the document loader).
    - Load Document: Use AzureAIDocumentIntelligenceLoader to load the document with specified API credentials and settings (remember to use pre-built layout as model and the latest API version)
    - Split Document: Split the loaded document using a custom advanced text splitter.
    - Store Document: Add the processed documents to a multimodal vector store (using the add_documents method).

In [ ]:
def advanced_text_splitter(documents):
    """Advanced text splitter for multimodal documents"""
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    import hashlib
    
    # Create splitter optimized for financial documents
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=1500,  # Larger chunks for tables/images
        chunk_overlap=300,
        separators=["\n\n", "\n", ". ", " ", ""]
    )
    
    split_docs = splitter.split_documents(documents)
    
    # Add unique IDs and enhance metadata
    for i, doc in enumerate(split_docs):
        content_hash = hashlib.md5(doc.page_content.encode()).hexdigest()
        doc.metadata['id'] = f"{content_hash}_{i}"
        
        # Determine content type
        if 'table' in doc.page_content.lower() or '|' in doc.page_content:
            doc.metadata['content_type'] = 'table'
        elif 'image' in doc.metadata.get('source', '').lower():
            doc.metadata['content_type'] = 'image'
        else:
            doc.metadata['content_type'] = 'text'
    
    return split_docs

def get_image_description(docs):
    """Extract image descriptions from document metadata"""
    context_parts = []
    
    for doc in docs:
        content = doc.page_content
        metadata = doc.metadata
        
        # Add content type information
        content_type = metadata.get('content_type', 'text')
        if content_type == 'image':
            content = f"[IMAGE DESCRIPTION]: {content}"
        elif content_type == 'table':
            content = f"[TABLE DATA]: {content}"
        
        context_parts.append(content)
    
    return "\n\n".join(context_parts)

def multimodal_prompt(context, question):
    """Create advanced multimodal prompt"""
    from langchain_core.prompts import ChatPromptTemplate
    
    prompt_template = ChatPromptTemplate.from_messages([
        ("system", f"""{system_prompt}
        
You are working with multimodal financial documents that may contain:
- Regular text content
- Table data (marked with [TABLE DATA])
- Image descriptions (marked with [IMAGE DESCRIPTION])

When answering questions:
1. Pay special attention to tables for numerical data
2. Use image descriptions for visual context
3. Combine information from multiple content types
4. If creating tables, use proper formatting

Context:
{context}"""),
        ("human", question)
    ])
    
    return prompt_template

# Index

# Index: Load files

# Get list of files in a local folder that start with "2023"
folder = "../..//data/fsi/pdf"
files = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f)) and f.startswith("2023")]
files = [os.path.join(folder, f) for f in files]

# For each file
for file in files:
    # Get the file name
    pdf_file_name = file.split("\\")[-1]
    # Index : Load the file and create a document
    print("Processing: ", file)

    loader = AzureAIDocumentIntelligenceLoader(
            file_path=file,
            api_key=AZURE_DOCUMENT_INTELLIGENCE_API_KEY,
            api_endpoint=AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT,
            api_model="prebuilt-layout",  # Use prebuilt-layout for multimodal support
            api_version=AZURE_DOCUMENT_INTELLIGENCE_API_VERSION,  # Latest API version
            analysis_features=[
                DocumentAnalysisFeature.OCR_HIGH_RESOLUTION,
                DocumentAnalysisFeature.FORMULAS,
                DocumentAnalysisFeature.STYLE_FONT
            ]
        )

    documents = loader.load()
    print(f"  Loaded {len(documents)} document sections")
    

    # Index: Split using advanced text splitter
    split_docs = advanced_text_splitter(documents)
    print(f"  Split into {len(split_docs)} chunks")
    
    # Index: Store in vector store
    vector_store.add_documents(split_docs)
    print(f"  ✅ Added {pdf_file_name} to vector store")
    
    

The current supported features are: ['ocrHighResolution']. Using other features may result in unexpected behavior.


Processing:  ../../data/fsi/pdf/2023 FY AMZN.pdf


ResourceNotFoundError: (404) Resource not found
Code: 404
Message: Resource not found

## Retrieve Phase

The next step is to create a retriever for the documents based on the user query.
You should use the following parameters:
- Search Type: Hybrid
- number of results: 20

In [ ]:
# Retrieve  (as_retriever)


## Generate Phase

The final step is to generate the answer using the RAG model.
We will create a Langchain chain with the following steps:
 - Retrieve the docs and get the image description if the doc matedata is an image (with get_image_description function - RunnableLambda), then pass the context and question (using RunnablePassthrough) to the next phase
 - Use the advanced multimodal Prompt function to append system messages, the context including the text, the image (if present) and the question - check RannableLambda method also here.
 - Use the OpenAI model to generate the answer
 - Parse the output and return the answer

In [ ]:
# Generate

# RAG pipeline


## Test the Solution

You can test the solution by providing a question and checking the answer generated by the RAG model (invoke the Langchain chain).

Try to get answer for the following questions:


In [ ]:
# Test the solution
for QUESTION in QUESTIONS:
    print(f"QUESTION: {QUESTION}")
    print(chain_multimodal_rag.invoke(QUESTION))
    print("--------------------------------------------------")